In [116]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize the neural network with one hidden layer.
        """
        # Weights and biases initialization
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01  # Input to hidden layer
        self.b1 = np.zeros((1, hidden_size))                      # Bias for hidden layer
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01  # Hidden to output layer
        self.b2 = np.zeros((1, output_size))                      # Bias for output layer

    def sigmoid(self, z):
        """
        Sigmoid activation function.
        """
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, a):
        """
        Derivative of the sigmoid activation function.
        """
        return a * (1 - a)

    def softmax(self, z):
        """
        Softmax activation function for output layer.
        """
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def compute_loss(self, y_true, y_pred):
        """
        Compute cross-entropy loss.
        """
        m = y_true.shape[0]  # Number of samples
        loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m  # Add small value for numerical stability
        return loss

    def train(self, X, y, epochs, learning_rate):
        """
        Train the neural network using gradient descent.
        """
        for epoch in range(epochs):
            # Forward pass
            z1 = np.dot(X, self.W1) + self.b1
            a1 = self.sigmoid(z1)
            z2 = np.dot(a1, self.W2) + self.b2
            a2 = self.softmax(z2)  # Output layer

            # Compute loss
            loss = self.compute_loss(y, a2)

            # Backward pass
            m = X.shape[0]  # Number of samples
            dz2 = a2 - y
            dW2 = np.dot(a1.T, dz2) / m
            db2 = np.sum(dz2, axis=0, keepdims=True) / m

            da1 = np.dot(dz2, self.W2.T)
            dz1 = da1 * self.sigmoid_derivative(a1)
            dW1 = np.dot(X.T, dz1) / m
            db1 = np.sum(dz1, axis=0, keepdims=True) / m

            # Update weights and biases
            self.W1 -= learning_rate * dW1
            self.b1 -= learning_rate * db1
            self.W2 -= learning_rate * dW2
            self.b2 -= learning_rate * db2

            # Print loss every 100 epochs
            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Predict probabilities for each class.
        """
        z1 = np.dot(X, self.W1) + self.b1
        a1 = self.sigmoid(z1)
        z2 = np.dot(a1, self.W2) + self.b2
        return self.softmax(z2)  # Probabilities for each class

In [118]:
import zipfile
import os
from PIL import Image
from sklearn.model_selection import train_test_split

In [120]:
def load_and_preprocess_images(image_folder, image_size):
    images = []
    labels = []
    class_names = sorted(os.listdir(image_folder))  # Sorted list of subfolder names

    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(image_folder, class_name)
        if not os.path.isdir(class_folder):
            print(f"Skipping non-folder item: {class_name}")
            continue

        print(f"Processing images in class folder: {class_folder}")
        for image_file in os.listdir(class_folder):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(class_folder, image_file)
                try:
                    img = Image.open(image_path).convert('L')  # Convert to grayscale
                    img = img.resize((image_size, image_size))  # Resize image
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    images.append(img_array.flatten())  # Flatten image
                    labels.append(class_index)  # Assign class label
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
            else:
                print(f"Skipping non-image file: {image_file}")
    
    print(f"Loaded {len(images)} images with {len(labels)} labels.")
    return np.array(images), np.array(labels)

In [84]:
!pip install opencv-python

In [122]:
import os
import zipfile
import cv2
from PIL import Image

In [124]:
def extract_images_and_labels(zip_path):
    images = []
    labels = []

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        list_of_files = zip_ref.namelist()

        for file_name in list_of_files:
            path_parts = file_name.split('/')
            if len(path_parts) > 1 and path_parts[-1].endswith('.png'):
                subfolder_name = path_parts[-2]  # Class name is the subfolder name
                
                # Read image
                in_bytes = zip_ref.read(file_name)
                img = cv2.imdecode(np.frombuffer(in_bytes, np.uint8), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (28, 28))  # Resize to 28x28
                img = img / 255.0  # Normalize pixel values

                images.append(img.flatten())  # Flatten the image
                labels.append(subfolder_name)  # Append the class name

    # Ensure we return the extracted data
    return np.array(images), np.array(labels)

In [126]:
zip_path = "C:/Users/maila/OneDrive/Desktop/Manas/task4/Train.zip"
X,y = extract_images_and_labels(zip_path)

In [128]:
print(f"Number of images: {X.shape[0]}")
print(f"Number of labels: {len(y)}")
print(f"Unique classes: {np.unique(y)}")

Number of images: 4000
Number of labels: 4000
Unique classes: ['Jade' 'James' 'Jane' 'Joel' 'Jovi']


In [130]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_numeric = le.fit_transform(y)
print(f"Encoded labels: {y_numeric}")

Encoded labels: [1 1 1 ... 4 4 4]


In [132]:
num_classes = len(np.unique(y_numeric))
y_one_hot = np.eye(num_classes)[y_numeric]

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

Training samples: 3200, Testing samples: 800


In [170]:
nn = NeuralNetwork(input_size=28*28, hidden_size=32, output_size=num_classes)
nn.train(X_train, y_train, epochs=1000, learning_rate=0.1)

Epoch 100/1000, Loss: 1.5773
Epoch 200/1000, Loss: 1.3480
Epoch 300/1000, Loss: 1.0824
Epoch 400/1000, Loss: 0.8525
Epoch 500/1000, Loss: 0.6782
Epoch 600/1000, Loss: 0.5638
Epoch 700/1000, Loss: 0.4859
Epoch 800/1000, Loss: 0.4293
Epoch 900/1000, Loss: 0.3863
Epoch 1000/1000, Loss: 0.3525


In [172]:
print(f"Shape of predictions: {predictions.shape}")
predicted_classes = np.argmax(predictions, axis=0) if len(predictions.shape) == 1 else np.argmax(predictions, axis=1)

Shape of predictions: (800, 5)


In [174]:
predictions = nn.predict(X_test)  # Ensure this outputs probabilities for each class
print(f"Shape of predictions: {predictions.shape}")

# Get predicted and true classes
if len(predictions.shape) == 1:  # Binary classification
    predicted_classes = (predictions >= 0.5).astype(int)
else:  # Multi-class classification
    predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

# Calculate accuracy
accuracy = np.mean(predicted_classes == true_classes)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Shape of predictions: (800, 5)
Test accuracy: 91.38%


In [216]:
#PREDICITING ON TEST.ZIP FILE

In [204]:
zip_path_2 = "C:/Users/maila/OneDrive/Desktop/Manas/task4/Test.zip"

In [206]:
X,y = extract_images_and_labels(zip_path_2)

In [208]:
print(f"Number of images: {X.shape[0]}")
print(f"Number of labels: {len(y)}")
print(f"Unique classes: {np.unique(y)}")

Number of images: 1000
Number of labels: 1000
Unique classes: ['Jade' 'James' 'Jane' 'Joel' 'Jovi']


In [210]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, categories='auto')
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))

In [212]:
predictions = nn.predict(X)

In [214]:
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_one_hot, axis=1))
print(f"Test accuracy: {accuracy * 100:.2f}%")

Test accuracy: 91.40%
